In [2]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import psycopg2
import seaborn as sns
from sqlalchemy import create_engine
from datetime import datetime

In [3]:
dbTable = 'gd_wTi_2021_08_07'

stDate = '2021-07-06'
endDate = '2021-08-06'

min_avgVol_30d = '140000'

orderBy = 'close_chngpct_24_tmro'

# Ov0_cutoff = 0.6

qualit_list = [ 
            'ticker',
            'open',
            'high',
            'low',
            'close',
            'timestamp',
            'date',
            # 'close_1wk_low',
            # 'close_1wk_high',
            # 'close_1mo_high',
            # 'close_1mo_low',
            # 'close_3mo_high',
            # 'close_3mo_low',
            # 'volume_1wk_low',
            # 'volume_1wk_high',
            # 'volume_1mo_low',
            # 'volume_1mo_high',
            'fib_pivot',
            'fib_s1',
            'fib_s2',
            'fib_s3',
            'fib_s4',
            'fib_r1',
            'fib_r2',
            'fib_r3',
            'fib_r4',
            'BB_up',
            'BB_mid',
            'BB_low',
            'EMA5',
            'EMA10',
            'EMA30',
            'EMA50',
            'EMA100',
            # 'EMA200',
            'SMA5',
            'SMA10',
            'SMA30',
            'SMA50',
            'SMA100',
            # 'SMA200',
            # 'HMA9',
            # 'tenkan',
            # 'kijun',
            # 'senkou_span_a',
            # 'senkou_span_b',
            'close_chngpct_24_tmro',
            # 'close_chngpct_48_tmro'
        ]

In [4]:
scanql = ( 
    '''
    SELECT * FROM "{dbTable}"
    WHERE "date" BETWEEN '{stDate}' AND '{endDate}'
    AND "avgVol_30d" > {min_avgVol_30d}
    ORDER BY "{orderBy}";
    '''
).format(dbTable=dbTable, stDate=stDate, endDate=endDate, min_avgVol_30d=min_avgVol_30d, orderBy=orderBy)


engine = ( 
    create_engine(
        'postgresql+psycopg2://postgres:s@unAsh*w3r@localhost/market'
    )
)
dbdf = pd.read_sql_query(sql=scanql, con=engine)

dbdf.drop(index=dbdf.loc[dbdf['close_chngpct_24_tmro'].isnull()].index, inplace=True)
dbdf.sort_values(by='close_chngpct_24_tmro', ascending=False, ignore_index=True, inplace=True)

dbdf.info(memory_usage='deep')

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 80580 entries, 0 to 80579
Columns: 220 entries, ticker to close_pctf_ATRonSMA10
dtypes: float64(216), int64(2), object(2)
memory usage: 143.8 MB


In [ ]:
dbdf.isnull().sum().sort_values(ascending=False).head(20)

In [ ]:
dropNulls_list = [
#     'chikou',
#     'VIp_div_pVIm',
#     'VIp_sub_pVIm',
#     # 'perf_52wk',
#     # 'close_52wk_low',
#     # 'close_52wk_high',
#     # 'close_6mo_high',
#     # 'perf_6mo',
#     # 'close_6mo_low',
]
# dbdf.drop(columns=dropNulls_list, inplace=True)

---

In [5]:
quant_cols = dbdf.columns.tolist()

for qualit_col in qualit_list:
    quant_cols.remove(qualit_col)

In [ ]:
dbdf[quant_cols].max().sort_values(ascending=False).head(10)

In [ ]:
dbdf[quant_cols].min().sort_values(ascending=True).head(10)

---
# | thirds solo

In [ ]:
''' thirds solox '''

In [ ]:
solo_df.sort_values(by='Ov0c', ascending=False).iloc[:50]

In [ ]:
solo_df.to_csv(f'/Users/clay/programming/finPal/finPal-app/scanql-tables/solotech_{stDate}_{endDate}.csv')

---
## | thirds (2 filters)

In [ ]:
# dbdf.loc[dbdf['close_chngpct_24_tmro'] > Ov0_cutoff, ['Ov0_cat']] = 'over'
# dbdf.loc[dbdf['close_chngpct_24_tmro'] <= Ov0_cutoff, ['Ov0_cat']] = 'under'

# Ov0_idx = dbdf.loc[dbdf['close_chngpct_24_tmro'] > Ov0_cutoff].index
# Uv0_idx = dbdf.loc[dbdf['close_chngpct_24_tmro'] <= Ov0_cutoff].index

In [6]:
thirds_rows = []
for techcol in quant_cols:

    print(f'{techcol} @ {datetime.strftime(datetime.now(), "%H:%M:%S")}')
    
    tech_q000 = dbdf[techcol].quantile(0.00)
    tech_q025 = dbdf[techcol].quantile(0.25)
    tech_q050 = dbdf[techcol].quantile(0.5)
    tech_q075 = dbdf[techcol].quantile(0.75)
    tech_q100 = dbdf[techcol].quantile(1.0)

    # under median
    subset_df = dbdf.loc[
        dbdf[techcol].between(
            left=tech_q000, 
            right=tech_q050
        )
    ]
    for techcol2 in quant_cols:
        # if techcol2 == techcol:
        #     continue
        # else:
        #     pass
        tech2_q000 = dbdf[techcol2].quantile(0.00)
        tech2_q050 = dbdf[techcol2].quantile(0.50)
        dubsubset_df = subset_df.loc[
            dbdf[techcol2].between( 
            left=tech2_q000,
            right=tech2_q050,
            )
        ]
        obsv = len(dubsubset_df)
        Ov0 = len(dubsubset_df.loc[dubsubset_df['close_chngpct_24_tmro'] > 0])
        Ov05 = len(dubsubset_df.loc[dubsubset_df['close_chngpct_24_tmro'] > 0.5])
        Ov1 = len(dubsubset_df.loc[dubsubset_df['close_chngpct_24_tmro'] > 1])
        if obsv > 0:
            Ov0c = (Ov0 / obsv)
            Ov05c = (Ov05 / obsv)
            Ov1c = (Ov1 / obsv)
        else:
            Ov0c = 0
        thirds_rows.append( 
            [ 
                techcol, 
                'q000_q050', 
                tech_q000, 
                tech_q050, 
                techcol2, 
                'q000_q050', 
                tech2_q000, 
                tech2_q050, 
                obsv,
                Ov0,
                Ov0c,
                Ov05,
                Ov05c,
                Ov1,
                Ov1c
            ]
        )
        tech2_q025 = dbdf[techcol2].quantile(0.25)
        tech2_q075 = dbdf[techcol2].quantile(0.75)
        dubsubset_df = subset_df.loc[
            dbdf[techcol2].between( 
            left=tech2_q025,
            right=tech2_q075,
            )
        ]
        obsv = len(dubsubset_df)
        Ov0 = len(dubsubset_df.loc[dubsubset_df['close_chngpct_24_tmro'] > 0])
        Ov05 = len(dubsubset_df.loc[dubsubset_df['close_chngpct_24_tmro'] > 0.5])
        Ov1 = len(dubsubset_df.loc[dubsubset_df['close_chngpct_24_tmro'] > 1])
        if obsv > 0:
            Ov0c = (Ov0 / obsv)
            Ov05c = (Ov05 / obsv)
            Ov1c = (Ov1 / obsv)
        else:
            Ov0c = 0
        thirds_rows.append( 
            [ 
                techcol, 
                'q000_q050', 
                tech_q000, 
                tech_q050, 
                techcol2, 
                'q025_q075', 
                tech2_q025, 
                tech2_q075, 
                obsv,
                Ov0,
                Ov0c,
                Ov05,
                Ov05c,
                Ov1,
                Ov1c
            ]
        )
        tech2_q100 = dbdf[techcol2].quantile(1.00)
        dubsubset_df = subset_df.loc[
            dbdf[techcol2].between( 
            left=tech2_q050,
            right=tech2_q100,
            )
        ]
        obsv = len(dubsubset_df)
        Ov0 = len(dubsubset_df.loc[dubsubset_df['close_chngpct_24_tmro'] > 0])
        Ov05 = len(dubsubset_df.loc[dubsubset_df['close_chngpct_24_tmro'] > 0.5])
        Ov1 = len(dubsubset_df.loc[dubsubset_df['close_chngpct_24_tmro'] > 1])
        if obsv > 0:
            Ov0c = (Ov0 / obsv)
            Ov05c = (Ov05 / obsv)
            Ov1c = (Ov1 / obsv)
        else:
            Ov0c = 0
        thirds_rows.append( 
            [ 
                techcol, 
                'q000_q050', 
                tech_q000, 
                tech_q050, 
                techcol2, 
                'q050_q100', 
                tech2_q050, 
                tech2_q100, 
                obsv,
                Ov0,
                Ov0c,
                Ov05,
                Ov05c,
                Ov1,
                Ov1c
            ]
        )
    # interquartile range
    subset_df = dbdf.loc[
        dbdf[techcol].between(
            left=tech_q025, 
            right=tech_q075
        )
    ]
    for techcol2 in quant_cols:
        # if techcol2 == techcol:
        #     continue
        # else:
        #     pass
        tech2_q000 = dbdf[techcol2].quantile(0.00)
        tech2_q050 = dbdf[techcol2].quantile(0.50)
        dubsubset_df = subset_df.loc[
            dbdf[techcol2].between( 
            left=tech2_q000,
            right=tech2_q050,
            )
        ]
        obsv = len(dubsubset_df)
        Ov0 = len(dubsubset_df.loc[dubsubset_df['close_chngpct_24_tmro'] > 0])
        Ov05 = len(dubsubset_df.loc[dubsubset_df['close_chngpct_24_tmro'] > 0.5])
        Ov1 = len(dubsubset_df.loc[dubsubset_df['close_chngpct_24_tmro'] > 1])
        if obsv > 0:
            Ov0c = (Ov0 / obsv)
            Ov05c = (Ov05 / obsv)
            Ov1c = (Ov1 / obsv)
        else:
            Ov0c = 0
        thirds_rows.append( 
            [ 
                techcol, 
                'q025_q075', 
                tech_q025, 
                tech_q075, 
                techcol2, 
                'q000_q050', 
                tech2_q000, 
                tech2_q050, 
                obsv,
                Ov0,
                Ov0c,
                Ov05,
                Ov05c,
                Ov1,
                Ov1c
            ]
        )
        tech2_q025 = dbdf[techcol2].quantile(0.25)
        tech2_q075 = dbdf[techcol2].quantile(0.75)
        dubsubset_df = subset_df.loc[
            dbdf[techcol2].between( 
            left=tech2_q025,
            right=tech2_q075,
            )
        ]
        obsv = len(dubsubset_df)
        Ov0 = len(dubsubset_df.loc[dubsubset_df['close_chngpct_24_tmro'] > 0])
        Ov05 = len(dubsubset_df.loc[dubsubset_df['close_chngpct_24_tmro'] > 0.5])
        Ov1 = len(dubsubset_df.loc[dubsubset_df['close_chngpct_24_tmro'] > 1])
        if obsv > 0:
            Ov0c = (Ov0 / obsv)
            Ov05c = (Ov05 / obsv)
            Ov1c = (Ov1 / obsv)
        else:
            Ov0c = 0
        thirds_rows.append( 
            [ 
                techcol, 
                'q025_q075', 
                tech_q025, 
                tech_q075, 
                techcol2, 
                'q025_q075', 
                tech2_q025, 
                tech2_q075, 
                obsv,
                Ov0,
                Ov0c,
                Ov05,
                Ov05c,
                Ov1,
                Ov1c
            ]
        )
        tech2_q100 = dbdf[techcol2].quantile(1.00)
        dubsubset_df = subset_df.loc[
            dbdf[techcol2].between( 
            left=tech2_q050,
            right=tech2_q100,
            )
        ]
        obsv = len(dubsubset_df)
        Ov0 = len(dubsubset_df.loc[dubsubset_df['close_chngpct_24_tmro'] > 0])
        Ov05 = len(dubsubset_df.loc[dubsubset_df['close_chngpct_24_tmro'] > 0.5])
        Ov1 = len(dubsubset_df.loc[dubsubset_df['close_chngpct_24_tmro'] > 1])
        if obsv > 0:
            Ov0c = (Ov0 / obsv)
            Ov05c = (Ov05 / obsv)
            Ov1c = (Ov1 / obsv)
        else:
            Ov0c = 0
        thirds_rows.append( 
            [ 
                techcol, 
                'q025_q075', 
                tech_q025, 
                tech_q075, 
                techcol2, 
                'q050_q100', 
                tech2_q050, 
                tech2_q100, 
                obsv,
                Ov0,
                Ov0c,
                Ov05,
                Ov05c,
                Ov1,
                Ov1c
            ]
        )
    # over median
    subset_df = dbdf.loc[
        dbdf[techcol].between(
            left=tech_q050, 
            right=tech_q100
        )
    ]
    for techcol2 in quant_cols:
        # if techcol2 == techcol:
        #     continue
        # else:
        #     pass
        tech2_q000 = dbdf[techcol2].quantile(0.00)
        tech2_q050 = dbdf[techcol2].quantile(0.50)
        dubsubset_df = subset_df.loc[
            dbdf[techcol2].between( 
            left=tech2_q000,
            right=tech2_q050,
            )
        ]
        obsv = len(dubsubset_df)
        Ov0 = len(dubsubset_df.loc[dubsubset_df['close_chngpct_24_tmro'] > 0])
        Ov05 = len(dubsubset_df.loc[dubsubset_df['close_chngpct_24_tmro'] > 0.5])
        Ov1 = len(dubsubset_df.loc[dubsubset_df['close_chngpct_24_tmro'] > 1])
        if obsv > 0:
            Ov0c = (Ov0 / obsv)
            Ov05c = (Ov05 / obsv)
            Ov1c = (Ov1 / obsv)
        else:
            Ov0c = 0
        thirds_rows.append( 
            [ 
                techcol, 
                'q050_q100', 
                tech_q050, 
                tech_q100, 
                techcol2, 
                'q000_q050', 
                tech2_q000, 
                tech2_q050, 
                obsv,
                Ov0,
                Ov0c,
                Ov05,
                Ov05c,
                Ov1,
                Ov1c
            ]
        )
        tech2_q025 = dbdf[techcol2].quantile(0.25)
        tech2_q075 = dbdf[techcol2].quantile(0.75)
        dubsubset_df = subset_df.loc[
            dbdf[techcol2].between( 
            left=tech2_q025,
            right=tech2_q075,
            )
        ]
        obsv = len(dubsubset_df)
        Ov0 = len(dubsubset_df.loc[dubsubset_df['close_chngpct_24_tmro'] > 0])
        Ov05 = len(dubsubset_df.loc[dubsubset_df['close_chngpct_24_tmro'] > 0.5])
        Ov1 = len(dubsubset_df.loc[dubsubset_df['close_chngpct_24_tmro'] > 1])
        if obsv > 0:
            Ov0c = (Ov0 / obsv)
            Ov05c = (Ov05 / obsv)
            Ov1c = (Ov1 / obsv)
        else:
            Ov0c = 0
        thirds_rows.append( 
            [ 
                techcol, 
                'q050_q100', 
                tech_q050, 
                tech_q100, 
                techcol2, 
                'q025_q075', 
                tech2_q025, 
                tech2_q075, 
                obsv,
                Ov0,
                Ov0c,
                Ov05,
                Ov05c,
                Ov1,
                Ov1c
            ]
        )
        tech2_q100 = dbdf[techcol2].quantile(1.00)
        dubsubset_df = subset_df.loc[
            dbdf[techcol2].between( 
            left=tech2_q050,
            right=tech2_q100,
            )
        ]
        obsv = len(dubsubset_df)
        Ov0 = len(dubsubset_df.loc[dubsubset_df['close_chngpct_24_tmro'] > 0])
        Ov05 = len(dubsubset_df.loc[dubsubset_df['close_chngpct_24_tmro'] > 0.5])
        Ov1 = len(dubsubset_df.loc[dubsubset_df['close_chngpct_24_tmro'] > 1])
        if obsv > 0:
            Ov0c = (Ov0 / obsv)
            Ov05c = (Ov05 / obsv)
            Ov1c = (Ov1 / obsv)
        else:
            Ov0c = 0
        thirds_rows.append( 
            [ 
                techcol, 
                'q050_q100', 
                tech_q050, 
                tech_q100, 
                techcol2, 
                'q050_q100', 
                tech2_q050, 
                tech2_q100, 
                obsv,
                Ov0,
                Ov0c,
                Ov05,
                Ov05c,
                Ov1,
                Ov1c
            ]
        )
    quant_cols.remove(techcol)

thirds_df = pd.DataFrame(
        data=thirds_rows, 
        columns=[ 
            'techcol',
            'techzone',
            'techleft',
            'techright',
            'tech2',
            'tech2zone',
            'tech2left',
            'tech2right',
            'obsv',
            'Ov0',
            'Ov0c', 
            'Ov05', 
            'Ov05c', 
            'Ov1', 
            'Ov1c'
        ]
)

        


volume @ 11:11:01
transactions @ 11:11:23
close_chngpct_48 @ 11:11:44
close_chngpct_24_yday @ 11:12:05
close_down3 @ 11:12:25
perf_1wk_sub_pchngpct_24 @ 11:12:56
close_chngpct_1wk_high @ 11:13:08
close_24_sub_n_pctf_1wk_high @ 11:13:29
close_24_sub_p_pctf_1wk_low @ 11:13:46
perf_1mo @ 11:14:02
close_chngpct_1mo_low @ 11:14:22
vwap_chngpct_48 @ 11:14:42
vwap_chngpct_24_yday @ 11:15:02
volume_chngpct_24 @ 11:15:22
volume_1wk_low @ 11:15:41
volume_chngpct_vlow @ 11:16:01
close_at_vlow_1wk @ 11:16:20
close_pctf_vlow_1wk @ 11:16:40
rvol @ 11:17:01
avgVol_10d_chngpct_48 @ 11:17:20
avgVol_30d @ 11:17:39
avgVol_30d_chngpct_48 @ 11:17:58
avgVol_10d_pctf_av30d @ 11:18:16
WOBV @ 11:18:35
WOBV_chngpct_48 @ 11:18:53
EFI @ 11:19:12
EFI_diff_n_24 @ 11:19:30
bop @ 11:19:43
bop_diff_n_24 @ 11:20:01
bop_sub_p_vhigh_1wk @ 11:20:13
bop_at_vlow_1wk @ 11:20:25
bop_sub_n_vlow_1wk @ 11:20:43
bop_EMA3 @ 11:20:55
bull_power @ 11:21:13
bullPwr_diff_p_24 @ 11:21:30
bearPwr_diff_p_24 @ 11:21:42
bullPwr_sub_p_bearP

In [7]:
thirds_df.info(memory_usage='deep')

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 122265 entries, 0 to 122264
Data columns (total 15 columns):
 #   Column      Non-Null Count   Dtype  
---  ------      --------------   -----  
 0   techcol     122265 non-null  object 
 1   techzone    122265 non-null  object 
 2   techleft    122265 non-null  float64
 3   techright   122265 non-null  float64
 4   tech2       122265 non-null  object 
 5   tech2zone   122265 non-null  object 
 6   tech2left   122265 non-null  float64
 7   tech2right  122265 non-null  float64
 8   obsv        122265 non-null  int64  
 9   Ov0         122265 non-null  int64  
 10  Ov0c        122265 non-null  float64
 11  Ov05        122265 non-null  int64  
 12  Ov05c       122265 non-null  float64
 13  Ov1         122265 non-null  int64  
 14  Ov1c        122265 non-null  float64
dtypes: float64(7), int64(4), object(4)
memory usage: 42.4 MB


In [8]:
# frmt_Ov0cutoff = f'{Ov0_cutoff}'.replace('.', '-')
# thirds_df.to_csv(f'/Users/clay/programming/finPal/finPal-app/scanql-tables/thirds_{stDate}_{endDate}_full_Ov{frmt_Ov0cutoff}_{min_avgVol_30d}.csv')

thirds_df.to_csv(f'/Users/clay/programming/finPal/finPal-app/scanql-tables/thirds_{stDate}_{endDate}_full_{min_avgVol_30d}.csv')

In [9]:
thirds_df.sort_values(by='Ov0c', ascending=False, inplace=True)

# frmt_Ov0cutoff = f'{Ov0_cutoff}'.replace('.', '-')
# thirds_df.loc[thirds_df['Ov0'] > 100].iloc[:250].to_csv(f'/Users/clay/programming/finPal/finPal-app/scanql-tables/thirds_{stDate}_{endDate}_top_Ov{frmt_Ov0cutoff}_{min_avgVol_30d}.csv')

thirds_df.loc[thirds_df['Ov0'] > 100].iloc[:250].to_csv(f'/Users/clay/programming/finPal/finPal-app/scanql-tables/thirds_{stDate}_{endDate}_top_{min_avgVol_30d}.csv')


---
## | pick2 - mix2 four filters

In [ ]:
pick1 = 'true_range'
pick1left = 0.82
pick1right = 569.55

pick2 = 'bullPwr_sub_n_bearPwr'
pick2left = -441.829686167372
pick2right = 0.569827789750434

p2m2_df = dbdf.loc[ 
    ( 
        dbdf[pick1].between( 
            left=pick1left,
            right=pick1right,
        )
    ) & ( 
        dbdf[pick2].between( 
            left=pick2left,
            right=pick2right
        )
    )
]

pick2mix2_rows = []

for techcol in quant_cols:

    print(techcol)

    tech_q000 = dbdf[techcol].quantile(0.00)
    tech_q025 = dbdf[techcol].quantile(0.25)
    tech_q050 = dbdf[techcol].quantile(0.5)
    tech_q075 = dbdf[techcol].quantile(0.75)
    tech_q100 = dbdf[techcol].quantile(1.0)

    # under median
    subset_df = p2m2_df.loc[
        p2m2_df[techcol].between(
            left=tech_q000, 
            right=tech_q050
        )
    ]
    for techcol2 in quant_cols:
        # if techcol2 == techcol:
        #     continue
        # else:
        #     pass
        tech2_q000 = dbdf[techcol2].quantile(0.00)
        tech2_q050 = dbdf[techcol2].quantile(0.50)
        dubsubset_df = subset_df.loc[
            p2m2_df[techcol2].between( 
            left=tech2_q000,
            right=tech2_q050,
            )
        ]
        obsv = len(dubsubset_df)
        Ov0 = len(dubsubset_df.loc[dubsubset_df['close_chngpct_24_tmro'] > 0])
        Ov05 = len(dubsubset_df.loc[dubsubset_df['close_chngpct_24_tmro'] > 0.5])
        Ov1 = len(dubsubset_df.loc[dubsubset_df['close_chngpct_24_tmro'] > 1])
        if obsv > 0:
            Ov0c = (Ov0 / obsv)
            Ov05c = (Ov05 / obsv)
            Ov1c = (Ov1 / obsv)
        else:
            Ov0c = 0
            Ov05c = 0
            Ov1c = 0
        pick2mix2_rows.append( 
            [ 
                techcol, 
                'q000_q050', 
                tech_q000, 
                tech_q050, 
                techcol2, 
                'q000_q050', 
                tech2_q000, 
                tech2_q050, 
                obsv,
                Ov0,
                Ov0c,
                Ov05,
                Ov05c,
                Ov1,
                Ov1c
            ]
        )
        tech2_q025 = dbdf[techcol2].quantile(0.25)
        tech2_q075 = dbdf[techcol2].quantile(0.75)
        dubsubset_df = subset_df.loc[
            p2m2_df[techcol2].between( 
            left=tech2_q025,
            right=tech2_q075,
            )
        ]
        obsv = len(dubsubset_df)
        Ov0 = len(dubsubset_df.loc[dubsubset_df['close_chngpct_24_tmro'] > 0])
        Ov05 = len(dubsubset_df.loc[dubsubset_df['close_chngpct_24_tmro'] > 0.5])
        Ov1 = len(dubsubset_df.loc[dubsubset_df['close_chngpct_24_tmro'] > 1])
        if obsv > 0:
            Ov0c = (Ov0 / obsv)
            Ov05c = (Ov05 / obsv)
            Ov1c = (Ov1 / obsv)
        else:
            Ov0c = 0
            Ov05c = 0
            Ov1c = 0
        pick2mix2_rows.append( 
            [ 
                techcol, 
                'q000_q050', 
                tech_q000, 
                tech_q050, 
                techcol2, 
                'q025_q075', 
                tech2_q025, 
                tech2_q075, 
                obsv,
                Ov0,
                Ov0c,
                Ov05,
                Ov05c,
                Ov1,
                Ov1c
            ]
        )
        tech2_q100 = dbdf[techcol2].quantile(1.00)
        dubsubset_df = subset_df.loc[
            p2m2_df[techcol2].between( 
            left=tech2_q050,
            right=tech2_q100,
            )
        ]
        obsv = len(dubsubset_df)
        Ov0 = len(dubsubset_df.loc[dubsubset_df['close_chngpct_24_tmro'] > 0])
        Ov05 = len(dubsubset_df.loc[dubsubset_df['close_chngpct_24_tmro'] > 0.5])
        Ov1 = len(dubsubset_df.loc[dubsubset_df['close_chngpct_24_tmro'] > 1])
        if obsv > 0:
            Ov0c = (Ov0 / obsv)
            Ov05c = (Ov05 / obsv)
            Ov1c = (Ov1 / obsv)
        else:
            Ov0c = 0
            Ov05c = 0
            Ov1c = 0
        pick2mix2_rows.append( 
            [ 
                techcol, 
                'q000_q050', 
                tech_q000, 
                tech_q050, 
                techcol2, 
                'q050_q100', 
                tech2_q050, 
                tech2_q100, 
                obsv,
                Ov0,
                Ov0c,
                Ov05,
                Ov05c,
                Ov1,
                Ov1c
            ]
        )
    subset_df = p2m2_df.loc[
        p2m2_df[techcol].between(
            left=tech_q025, 
            right=tech_q075
        )
    ]
    for techcol2 in quant_cols:
        # if techcol2 == techcol:
        #     continue
        # else:
        #     pass
        tech2_q000 = dbdf[techcol2].quantile(0.00)
        tech2_q050 = dbdf[techcol2].quantile(0.50)
        dubsubset_df = subset_df.loc[
            p2m2_df[techcol2].between( 
            left=tech2_q000,
            right=tech2_q050,
            )
        ]
        obsv = len(dubsubset_df)
        Ov0 = len(dubsubset_df.loc[dubsubset_df['close_chngpct_24_tmro'] > 0])
        Ov05 = len(dubsubset_df.loc[dubsubset_df['close_chngpct_24_tmro'] > 0.5])
        Ov1 = len(dubsubset_df.loc[dubsubset_df['close_chngpct_24_tmro'] > 1])
        if obsv > 0:
            Ov0c = (Ov0 / obsv)
            Ov05c = (Ov05 / obsv)
            Ov1c = (Ov1 / obsv)
        else:
            Ov0c = 0
            Ov05c = 0
            Ov1c = 0
        pick2mix2_rows.append( 
            [ 
                techcol, 
                'q025_q075', 
                tech_q025, 
                tech_q075, 
                techcol2, 
                'q000_q050', 
                tech2_q000, 
                tech2_q050, 
                obsv,
                Ov0,
                Ov0c,
                Ov05,
                Ov05c,
                Ov1,
                Ov1c
            ]
        )
        tech2_q025 = dbdf[techcol2].quantile(0.25)
        tech2_q075 = dbdf[techcol2].quantile(0.75)
        dubsubset_df = subset_df.loc[
            p2m2_df[techcol2].between( 
            left=tech2_q025,
            right=tech2_q075,
            )
        ]
        obsv = len(dubsubset_df)
        Ov0 = len(dubsubset_df.loc[dubsubset_df['close_chngpct_24_tmro'] > 0])
        Ov05 = len(dubsubset_df.loc[dubsubset_df['close_chngpct_24_tmro'] > 0.5])
        Ov1 = len(dubsubset_df.loc[dubsubset_df['close_chngpct_24_tmro'] > 1])
        if obsv > 0:
            Ov0c = (Ov0 / obsv)
            Ov05c = (Ov05 / obsv)
            Ov1c = (Ov1 / obsv)
        else:
            Ov0c = 0
            Ov05c = 0
            Ov1c = 0
        pick2mix2_rows.append( 
            [ 
                techcol, 
                'q025_q075', 
                tech_q025, 
                tech_q075, 
                techcol2, 
                'q025_q075', 
                tech2_q025, 
                tech2_q075, 
                obsv,
                Ov0,
                Ov0c,
                Ov05,
                Ov05c,
                Ov1,
                Ov1c
            ]
        )
        tech2_q100 = dbdf[techcol2].quantile(1.00)
        dubsubset_df = subset_df.loc[
            p2m2_df[techcol2].between( 
            left=tech2_q050,
            right=tech2_q100,
            )
        ]
        obsv = len(dubsubset_df)
        Ov0 = len(dubsubset_df.loc[dubsubset_df['close_chngpct_24_tmro'] > 0])
        Ov05 = len(dubsubset_df.loc[dubsubset_df['close_chngpct_24_tmro'] > 0.5])
        Ov1 = len(dubsubset_df.loc[dubsubset_df['close_chngpct_24_tmro'] > 1])
        if obsv > 0:
            Ov0c = (Ov0 / obsv)
            Ov05c = (Ov05 / obsv)
            Ov1c = (Ov1 / obsv)
        else:
            Ov0c = 0
            Ov05c = 0
            Ov1c = 0
        pick2mix2_rows.append( 
            [ 
                techcol, 
                'q025_q075', 
                tech_q025, 
                tech_q075, 
                techcol2, 
                'q050_q100', 
                tech2_q050, 
                tech2_q100, 
                obsv,
                Ov0,
                Ov0c,
                Ov05,
                Ov05c,
                Ov1,
                Ov1c
            ]
        )
    subset_df = p2m2_df.loc[
        p2m2_df[techcol].between(
            left=tech_q050, 
            right=tech_q100
        )
    ]
    for techcol2 in quant_cols:
        # if techcol2 == techcol:
        #     continue
        # else:
        #     pass
        tech2_q000 = dbdf[techcol2].quantile(0.00)
        tech2_q050 = dbdf[techcol2].quantile(0.50)
        dubsubset_df = subset_df.loc[
            p2m2_df[techcol2].between( 
            left=tech2_q000,
            right=tech2_q050,
            )
        ]
        obsv = len(dubsubset_df)
        Ov0 = len(dubsubset_df.loc[dubsubset_df['close_chngpct_24_tmro'] > 0])
        Ov05 = len(dubsubset_df.loc[dubsubset_df['close_chngpct_24_tmro'] > 0.5])
        Ov1 = len(dubsubset_df.loc[dubsubset_df['close_chngpct_24_tmro'] > 1])
        if obsv > 0:
            Ov0c = (Ov0 / obsv)
            Ov05c = (Ov05 / obsv)
            Ov1c = (Ov1 / obsv)
        else:
            Ov0c = 0
            Ov05c = 0
            Ov1c = 0
        pick2mix2_rows.append( 
            [ 
                techcol, 
                'q050_q100', 
                tech_q050, 
                tech_q100, 
                techcol2, 
                'q000_q050', 
                tech2_q000, 
                tech2_q050, 
                obsv,
                Ov0,
                Ov0c,
                Ov05,
                Ov05c,
                Ov1,
                Ov1c
            ]
        )
        tech2_q025 = dbdf[techcol2].quantile(0.25)
        tech2_q075 = dbdf[techcol2].quantile(0.75)
        dubsubset_df = subset_df.loc[
            p2m2_df[techcol2].between( 
            left=tech2_q025,
            right=tech2_q075,
            )
        ]
        obsv = len(dubsubset_df)
        Ov0 = len(dubsubset_df.loc[dubsubset_df['close_chngpct_24_tmro'] > 0])
        Ov05 = len(dubsubset_df.loc[dubsubset_df['close_chngpct_24_tmro'] > 0.5])
        Ov1 = len(dubsubset_df.loc[dubsubset_df['close_chngpct_24_tmro'] > 1])
        if obsv > 0:
            Ov0c = (Ov0 / obsv)
            Ov05c = (Ov05 / obsv)
            Ov1c = (Ov1 / obsv)
        else:
            Ov0c = 0
            Ov05c = 0
            Ov1c = 0
        pick2mix2_rows.append( 
            [ 
                techcol, 
                'q050_q100', 
                tech_q050, 
                tech_q100, 
                techcol2, 
                'q025_q075', 
                tech2_q025, 
                tech2_q075, 
                obsv,
                Ov0,
                Ov0c,
                Ov05,
                Ov05c,
                Ov1,
                Ov1c
            ]
        )
        tech2_q100 = dbdf[techcol2].quantile(1.00)
        dubsubset_df = subset_df.loc[
            p2m2_df[techcol2].between( 
            left=tech2_q050,
            right=tech2_q100,
            )
        ]
        obsv = len(dubsubset_df)
        Ov0 = len(dubsubset_df.loc[dubsubset_df['close_chngpct_24_tmro'] > 0])
        Ov05 = len(dubsubset_df.loc[dubsubset_df['close_chngpct_24_tmro'] > 0.5])
        Ov1 = len(dubsubset_df.loc[dubsubset_df['close_chngpct_24_tmro'] > 1])
        if obsv > 0:
            Ov0c = (Ov0 / obsv)
            Ov05c = (Ov05 / obsv)
            Ov1c = (Ov1 / obsv)
        else:
            Ov0c = 0
            Ov05c = 0
            Ov1c = 0
        pick2mix2_rows.append( 
            [ 
                techcol, 
                'q050_q100', 
                tech_q050, 
                tech_q100, 
                techcol2, 
                'q050_q100', 
                tech2_q050, 
                tech2_q100, 
                obsv,
                Ov0,
                Ov0c,
                Ov05,
                Ov05c,
                Ov1,
                Ov1c
            ]
        )
    quant_cols.remove(techcol)

pick2mix2_df = pd.DataFrame(
        data=pick2mix2_rows, 
        columns=[ 
            'techcol',
            'techzone',
            'techleft',
            'techright',
            'tech2',
            'tech2zone',
            'tech2left',
            'tech2right',
            'obsv',
            'Ov0',
            'Ov0c',
            'Ov05',
            'Ov05c',
            'Ov1',
            'Ov1c'
        ]
)

pick2mix2_df['pick1'] = pick1
pick2mix2_df['pick1left'] = pick1left
pick2mix2_df['pick1right'] = pick1right
pick2mix2_df['pick2'] = pick2
pick2mix2_df['pick2left'] = pick2left
pick2mix2_df['pick2right'] = pick2right

pick2mix2_df.sort_values(by='Ov0c', ascending=False, inplace=True)
pick2mix2_df.head(50)


In [ ]:
pick2mix2_df.loc[pick2mix2_df['Ov0'] > 100].head(50)

In [ ]:
pick2mix2_df.loc[pick2mix2_df['Ov0'] > 100].iloc[:250].to_csv(f'/Users/clay/programming/finPal/finPal-app/scanql-tables/quardo_{pick1}_{pick2}_{stDate}_{endDate}.csv')

---